In [105]:
from IPython.core.display import HTML
with open('../style.css', 'r') as file:
    css = file.read()
HTML(css)

# The Zebra Puzzle

The following puzzle appeared in the magazine *Life International* on the 17th of December in the year 1962:
<ol>
    <li>There are five houses.</li>
    <li>The Englishman lives in the red house.</li>
    <li>The Spaniard owns the dog.</li>
    <li>Coffee is drunk in the green house.</li>
    <li>The Ukrainian drinks tea.</li>
    <li>The green house is immediately to the right of the ivory house.</li>
    <li>The Old Gold smoker owns snails.</li>
    <li>Kools are smoked in the yellow house.</li>
    <li>Milk is drunk in the middle house.</li>
    <li>The Norwegian lives in the first house.</li>
    <li>The man who smokes Chesterfields lives in the house next to the man with the fox.</li>
    <li>Kools are smoked in the house next to the house where the horse is kept.</li>
    <li>The Lucky Strike smoker drinks orange juice.</li>
    <li>The Japanese smokes Parliaments.</li>
    <li>The Norwegian lives next to the blue house.</li>
</ol>
Furthermore, each of the five houses is painted in a different colour, their inhabitants are of different nationalities, own different pets, drink different beverages, and smoke different brands of cigarettes.

Your task is to write a program that answers the following questions: 
<ul>
    <li><b>Who drinks water?</b></li>
    <li><b>Who owns the zebra?</b></li>
</ul>

## Choosing the Appropriate Variables

In order to solve this problem we use the following propositional variables:  
<ol>
<li> For $i \in \{1,\cdots,5\}$ the variable $\texttt{English}\langle i \rangle$ expresses the fact
     that the Englishman lives in house number $i$.
     The remaining nationalities are coded using the variables
     $\texttt{Spanish}\langle i \rangle$, $\texttt{Ukrainian}\langle i \rangle$, $\texttt{Norwegian}\langle i \rangle$,      and $\texttt{Japanese}\langle i \rangle$.
     </li>
<li> For $i \in \{1,\cdots,5\}$ the variable $\mathtt{Red}\langle i \rangle$ expresses that house number $i$ is red.
     We use the variables $\texttt{Green}\langle i \rangle$, $\texttt{Ivory}\langle i \rangle$, $\texttt{Yellow}\langle i \rangle$, and $\texttt{Blue}\langle i \rangle$ to encode the remaining colors.
     </li>
<li> For $i \in \{1,\cdots,5\}$ the variable $\texttt{OldGold}\langle i \rangle$ expresses that the inhabitant of house      number $i$ smokes cigarettes of the brand "Old Gold".
     We use the variables $\texttt{Kools}\langle i \rangle$, $\texttt{Chesterfields}\langle i \rangle$,                      $\texttt{LuckyStrike}\langle i \rangle$, and $\texttt{Parliaments}\langle i \rangle$ to encode the remaining
     cigarette brands.
     </li>
<li> For $i \in \{1,\cdots,5\}$ the variable $\texttt{Dog}\langle i \rangle$ expresses the fact that the inhabitant 
     of house number $i$ keeps a dog as his pet. We use the variables
     $\texttt{Snails}\langle i \rangle$, $\texttt{Fox}\langle i \rangle$, $\texttt{Horse}\langle i \rangle$, and
     $\texttt{Zebra}\langle i \rangle$ to encode the remaining pets.
     </li>
<li> For $i \in \{1,\cdots,5\}$ the variable $\texttt{Coffee}\langle i \rangle$ expresses the fact the the 
     inhabitant of house number $i$ drinks coffee.  We use the variables
     $\texttt{Milk}\langle i \rangle$, $\texttt{OrangeJuice}\langle i \rangle$, $\texttt{Tea}\langle i \rangle$, 
     and $\texttt{Water}\langle i \rangle$ to encode the remaining drinks.
     </li>
<ol>
 
<p> 
We are using the angular brackets "$\langle$" and "$\rangle$" because our parser for propositional logic accepts these symbols as part of variable names.  The parser would be confused if we would use the normal parenthesis.
</p>

## Importing the Necessary Modules

Our goal is to solve this puzzle by first coding it as a solvability problem of propositional logic and then to solve the resulting set of clauses using the algorithm of Davis and Putnam.

In [106]:
import davisPutnam as dp

In order to be able to transform formulas from propositional logic into sets of clauses we import the module <tt>cnf</tt> which implements the function <tt>normalize</tt> that takes a formula and transforms it into a set of clauses.

In [107]:
import cnf

In order to write formulas conveniently, we use the parser for propositional logic.

In [108]:
import propLogParser as plp

Using the parser and the module <tt>cnf</tt> we can impement a function $\texttt{parseCNF}(s)$ that takes a string $s$ representing a formula and transforms $s$ into an equivalent set of clauses.

In [109]:
def parseCNF(s):
    nestedTuple = plp.LogicParser(s).parse()
    Clauses     = cnf.normalize(nestedTuple)
    return Clauses

In [110]:
parseCNF('p ∨ r → q')

{frozenset({('¬', 'r'), 'q'}), frozenset({('¬', 'p'), 'q'})}

## Auxiliary Functions

In order to succinctly express the constraints that all houses have different colours, the inhabitants have different nationalities etc., it is convenient to implement a function $\texttt{atMostOne}(V)$ that takes a set of variables $V$ and returns a set of formulas that is true if and only if all the variables from $V$ have different values.

In [111]:
def atMostOne(V):
    return { frozenset({('¬',p), ('¬', q)}) for p in V
                                            for q in V 
                                            if  p != q 
           }

Given a name $f$ and an index $i \in\{1,2,3,4,5\}$, the function $\texttt{var}(i)$ creates the string 
$f\langle i \rangle$, e.g. the call <tt>var("Japanese", 2)</tt> returns the following string:

<tt>Japanese$\langle$2$\rangle$</tt>.

In [112]:
def var(f, i):
    return f + "<" + str(i) + ">" 

In [113]:
var("Japanese", 2)

'Japanese<2>'

A call of the form $\texttt{somewhere}(x)$ will return a clause that specifies that the person with property $x$ has to live in one of the houses from $1$ to $5$.

In [114]:
def somewhere(x):
    return frozenset({ var(x, i) for i in range(1, 5+1) })

In [115]:
somewhere("a")

frozenset({'a<1>', 'a<2>', 'a<3>', 'a<4>', 'a<5>'})

Given an exclusive set of properties $S$ and a house number $i$, the function $\texttt{atMostOneAt}(S, i)$ returns a set of clauses that specifies that the person living in house number $i$ has at most one of the properties from the set $S$.  For example, if 
$S = \{\texttt{"Japanese"}, \texttt{"English"}, \texttt{"Spanish"}, \texttt{"Norwegian"}, \texttt{"Ukrainian"}\}$, 
then $\texttt{atMostOne}(S, 3)$ specifies that the inhabitant of house number 3 has at most one of the nationalities from the set $S$.

In [116]:
def atMostOneAt(S, i):
    return atMostOne({ var(x, i) for x in S })    

In [117]:
atMostOneAt({"A", "B", "C"}, 1)

{frozenset({('¬', 'B<1>'), ('¬', 'C<1>')}),
 frozenset({('¬', 'A<1>'), ('¬', 'C<1>')}),
 frozenset({('¬', 'A<1>'), ('¬', 'B<1>')})}

Implement a function $\texttt{onePerHouse}(S)$ which could be called as follows:
$$\texttt{onePerHouse}(\{\texttt{"Japanese"},
       \texttt{"English"}, 
       \texttt{"Spanish"}, \texttt{"Norwegian"}, 
       \texttt{"Ukrainian"}\})
$$
This function would create a set of clauses that expresses that there has to be a house where the Japanese lives, a house where the English lives, a house where the Spanish lives, a house where the Norwegian lives, and a house
where the Ukrainian lives.  Furthermore, the set of clauses would create clauses that express that these five persons live in **different** houses.

When implementing this function, you should use the functions <tt>somewhere</tt> and <tt>atMostOne</tt>.

In [118]:
def onePerHouse(S):
    result = set()
    
    atMost = set()
        
    for i in range(1, 5+1):
        atMost |= atMostOneAt(S, i)
    
    result |= atMost
    
    for s in S:
        
        result |= { somewhere(s) }
        
    
    return result

In [119]:
onePerHouse({"A", "B", "C", "D", "E"})

{frozenset({('¬', 'A<3>'), ('¬', 'E<3>')}),
 frozenset({('¬', 'A<4>'), ('¬', 'C<4>')}),
 frozenset({('¬', 'B<1>'), ('¬', 'E<1>')}),
 frozenset({('¬', 'A<4>'), ('¬', 'E<4>')}),
 frozenset({('¬', 'A<1>'), ('¬', 'C<1>')}),
 frozenset({('¬', 'B<3>'), ('¬', 'E<3>')}),
 frozenset({'B<1>', 'B<2>', 'B<3>', 'B<4>', 'B<5>'}),
 frozenset({('¬', 'B<2>'), ('¬', 'E<2>')}),
 frozenset({('¬', 'A<1>'), ('¬', 'E<1>')}),
 frozenset({('¬', 'C<5>'), ('¬', 'D<5>')}),
 frozenset({'D<1>', 'D<2>', 'D<3>', 'D<4>', 'D<5>'}),
 frozenset({('¬', 'C<4>'), ('¬', 'D<4>')}),
 frozenset({('¬', 'A<5>'), ('¬', 'E<5>')}),
 frozenset({('¬', 'A<3>'), ('¬', 'D<3>')}),
 frozenset({('¬', 'B<3>'), ('¬', 'C<3>')}),
 frozenset({('¬', 'B<2>'), ('¬', 'D<2>')}),
 frozenset({('¬', 'A<4>'), ('¬', 'B<4>')}),
 frozenset({('¬', 'A<2>'), ('¬', 'C<2>')}),
 frozenset({('¬', 'A<2>'), ('¬', 'D<2>')}),
 frozenset({('¬', 'B<5>'), ('¬', 'E<5>')}),
 frozenset({('¬', 'A<3>'), ('¬', 'C<3>')}),
 frozenset({('¬', 'A<2>'), ('¬', 'E<2>')}),
 frozenset({

Given to properties $a$ and $b$ the function $\texttt{sameHouse}(a, b)$ computes a set of clauses that specifies that if the inhabitant of house number $i$ has the property $a$, then he also has the property $b$ and vice versa.  For example, $\texttt{sameHouse}(\texttt{"Japanese"}, \texttt{"Dog"})$ specifies that the Japanese guy keeps a dog.

In [120]:
def sameHouse(a, b):
    return { C for i in range(1,5+1)
               for C in parseCNF(f"{var(a, i)} ↔ {var(b, i)}") 
           }

In [121]:
sameHouse("Red", "Tea")

{frozenset({('¬', 'Tea<1>'), 'Red<1>'}),
 frozenset({('¬', 'Red<1>'), 'Tea<1>'}),
 frozenset({('¬', 'Red<5>'), 'Tea<5>'}),
 frozenset({('¬', 'Tea<4>'), 'Red<4>'}),
 frozenset({('¬', 'Red<2>'), 'Tea<2>'}),
 frozenset({('¬', 'Tea<5>'), 'Red<5>'}),
 frozenset({('¬', 'Tea<3>'), 'Red<3>'}),
 frozenset({('¬', 'Tea<2>'), 'Red<2>'}),
 frozenset({('¬', 'Red<4>'), 'Tea<4>'}),
 frozenset({('¬', 'Red<3>'), 'Tea<3>'})}

Given to properties $a$ and $b$ the function $\texttt{nextTo}(a, b)$ computes a set of clauses that specifies that the inhabitants with properties $a$ and $b$ are direct neighbours.  For example, $\texttt{nextTo}(\texttt{'Japanese'}, \texttt{'Dog'})$ specifies that the Japanese guy lives next to the guy who keeps a dog.

In [141]:
def nextTo(a, b):
    result = set()
    
    for i in range(1, 5+1):
        if i != 1 and i != 5:
            result |= parseCNF(f"{var(a, i)} → {var(b, i+1)} ∨ {var(b, i-1)}") 
        
        
        elif i == 1:
            result |= parseCNF(f"{var(a, i)} → {var(b, i+1)}")
            
        elif i == 5:
            result |= parseCNF(f"{var(a, i)} → {var(b, i-1)}")
    
    return result

In [123]:
nextTo('A', 'B')

{frozenset({('¬', 'A<1>'), 'B<2>'}),
 frozenset({('¬', 'A<3>'), 'B<2>', 'B<4>'}),
 frozenset({('¬', 'A<2>'), 'B<1>', 'B<3>'}),
 frozenset({('¬', 'A<5>'), 'B<4>'}),
 frozenset({('¬', 'A<4>'), 'B<3>', 'B<5>'})}

Given to properties $a$ and $b$ the function $\texttt{leftTo}(a, b)$ computes a list of clauses that specifies that the inhabitants with properties $a$ lives in the house to the left of the inhabitant who has property $b$.  For example, $\texttt{leftTo}(\texttt{'Japanese'}, \texttt{'Dog'})$ specifies that the Japanese guy lives in the house to the left of the house where there is a dog.

In [124]:
def leftTo(a, b):
    result = set()
    
    for i in range(1, 4+1):
        
        result |= parseCNF(f"{var(a, i)} → {var(b, i+1)}")

    result |= parseCNF(f"¬{var(a, 5)}")
    
    return result

In [125]:
leftTo('A', 'B')

{frozenset({('¬', 'A<1>'), 'B<2>'}),
 frozenset({('¬', 'A<3>'), 'B<4>'}),
 frozenset({('¬', 'A<4>'), 'B<5>'}),
 frozenset({('¬', 'A<5>')}),
 frozenset({('¬', 'A<2>'), 'B<3>'})}

In [126]:
Nations  = { "English", "Spanish", "Ukrainian", "Norwegian", "Japanese" }
Drinks   = { "Coffee", "Tea", "Milk", "OrangeJuice", "Water" }
Pets     = { "Dog", "Snails", "Horse", "Fox", "Zebra" }
Brands   = { "LuckyStrike", "Parliaments", "Kools", "Chesterfields", "OldGold" }
Colours  = { "Red", "Green", "Ivory", "Yellow", "Blue" }

In [127]:
def allClauses():
    # Every house has exactly one inhabitant.  This inhabitant has exactly one
    # nationality, one pet, smokes one brand of cigarettes, and has one type
    # of drink.  Furthermore, every house has exactly one color.
    Clauses  = onePerHouse(Nations)
    Clauses |= onePerHouse(Drinks)
    Clauses |= onePerHouse(Pets)
    Clauses |= onePerHouse(Brands)
    Clauses |= onePerHouse(Colours)
    # The Englishman lives in the red house.
    Clauses |= sameHouse("English", "Red")
    # The Spaniard owns the dog.
    Clauses |= sameHouse("Spanish", "Dog")
    # Coffee is drunk in the green house.
    Clauses |= sameHouse("Coffee", "Green")
    # The Ukrainian drinks tea.
    Clauses |= sameHouse("Ukrainian", "Tea")
    # The green house is immediately to the right of the ivory house.
    Clauses |= leftTo("Ivory", "Green")
    # The Old Gold smoker owns snails.
    Clauses |= sameHouse("OldGold", "Snails")
    # Kools are smoked in the yellow house.
    Clauses |= sameHouse("Kools", "Yellow")
    # Milk is drunk in the middle house.
    Clauses |= parseCNF(f"{var('Milk', 3)}") 
    # The Norwegian lives in the first house.
    Clauses |= parseCNF(f"{var('Norwegian', 1)}") 
    # The man who smokes Chesterfields lives in the house next 
    # to the man with the fox.
    Clauses |= nextTo("Chesterfields", "Fox")
    # Kools are smoked in the house next to the house where the horse is kept.
    Clauses |= nextTo("Kools", "Horse")
    # The Lucky Strike smoker drinks orange juice.
    Clauses |= sameHouse("LuckyStrike", "OrangeJuice")
    # The Japanese smokes Parliaments.
    Clauses |= sameHouse("Japanese", "Parliaments")
    # The Norwegian lives next to the blue house.
    Clauses |= nextTo("Norwegian", "Blue")
    return Clauses

In [128]:
Clauses = allClauses()
Clauses

{frozenset({('¬', 'Kools<2>'), ('¬', 'Parliaments<2>')}),
 frozenset({('¬', 'Kools<5>'), ('¬', 'LuckyStrike<5>')}),
 frozenset({('¬', 'Green<3>'), 'Coffee<3>'}),
 frozenset({('¬', 'Coffee<2>'), 'Green<2>'}),
 frozenset({('¬', 'Coffee<3>'), 'Green<3>'}),
 frozenset({('¬', 'English<1>'), ('¬', 'Ukrainian<1>')}),
 frozenset({('¬', 'Dog<1>'), ('¬', 'Fox<1>')}),
 frozenset({'Parliaments<1>',
            'Parliaments<2>',
            'Parliaments<3>',
            'Parliaments<4>',
            'Parliaments<5>'}),
 frozenset({('¬', 'Ukrainian<1>'), 'Tea<1>'}),
 frozenset({('¬', 'Blue<1>'), ('¬', 'Red<1>')}),
 frozenset({('¬', 'Kools<2>'), 'Yellow<2>'}),
 frozenset({('¬', 'Green<3>'), ('¬', 'Red<3>')}),
 frozenset({('¬', 'Norwegian<2>'), 'Blue<1>', 'Blue<3>'}),
 frozenset({('¬', 'Blue<1>'), ('¬', 'Ivory<1>')}),
 frozenset({('¬', 'Chesterfields<4>'), 'Fox<3>', 'Fox<5>'}),
 frozenset({('¬', 'Dog<4>'), ('¬', 'Fox<4>')}),
 frozenset({('¬', 'Chesterfields<4>'), ('¬', 'LuckyStrike<4>')}),
 frozenset(

There should be 396 clauses.

In [129]:
len(Clauses)

377

In [142]:
def solve():
    Clauses = allClauses()
    return dp.solve(Clauses, set())

Solving the problem takes about 0.1 seconds on my computer.

In [143]:
%%time
Solution = solve()
Solution

CPU times: user 1.34 s, sys: 0 ns, total: 1.34 s
Wall time: 1.35 s


{frozenset({('¬', 'Tea<5>')}),
 frozenset({('¬', 'Spanish<3>')}),
 frozenset({('¬', 'Red<1>')}),
 frozenset({('¬', 'OrangeJuice<1>')}),
 frozenset({('¬', 'Fox<3>')}),
 frozenset({('¬', 'Fox<2>')}),
 frozenset({'Chesterfields<2>'}),
 frozenset({('¬', 'Dog<1>')}),
 frozenset({('¬', 'Parliaments<3>')}),
 frozenset({('¬', 'Japanese<2>')}),
 frozenset({'Japanese<5>'}),
 frozenset({('¬', 'Zebra<4>')}),
 frozenset({('¬', 'Norwegian<3>')}),
 frozenset({('¬', 'Snails<2>')}),
 frozenset({'Horse<2>'}),
 frozenset({('¬', 'Kools<4>')}),
 frozenset({'Fox<1>'}),
 frozenset({('¬', 'Green<2>')}),
 frozenset({('¬', 'Horse<1>')}),
 frozenset({('¬', 'Zebra<1>')}),
 frozenset({'Green<5>'}),
 frozenset({('¬', 'LuckyStrike<2>')}),
 frozenset({('¬', 'Norwegian<4>')}),
 frozenset({('¬', 'Green<3>')}),
 frozenset({('¬', 'Snails<4>')}),
 frozenset({'Red<3>'}),
 frozenset({('¬', 'Green<4>')}),
 frozenset({('¬', 'Red<2>')}),
 frozenset({'Parliaments<5>'}),
 frozenset({('¬', 'Milk<5>')}),
 frozenset({('¬', 'Milk<2>

## Functions to PrettyPrint the Solution

In [132]:
def arb(S):
    for x in S:
        return x

In [133]:
def extractAssignment(Solution):
    Assignment = {}
    for Unit in Solution:
        Literal = arb(Unit)
        if isinstance(Literal, str):
            number = int(Literal[-2])
            name   = Literal[:-3]
            Assignment[name] = number
    return Assignment

In [134]:
def showHTML(Solution):
    Assignment = extractAssignment(Solution)
    result  = '<table style="border:2px solid blue">\n'
    result += '<tr>'
    for name in ['House', 'Nationality',  'Drink', 'Animal', 'Brand', 'Colour']:
        result += '<th style="color:gold; background-color:blue">' + name + '</th>'
    result += '</tr>\n'
    for chair in range(1, 5+1):
        result += '<tr><td style="border:1px solid green">' + str(chair) + '</td>'
        for Class in [Nations, Drinks, Pets, Brands, Colours]:
            for x in Class:
                if Assignment[x] == chair:
                    result += '<td  style="border:1px solid green">' + x + '</td>'
        result += '</tr>\n'
    result += '</table>'
    display(HTML(result))

In [135]:
showHTML(Solution)

House,Nationality,Drink,Animal,Brand,Colour
1,Norwegian,Water,Fox,Kools,Yellow
2,Ukrainian,Tea,Horse,Chesterfields,Blue
3,English,Milk,Snails,OldGold,Red
4,Spanish,OrangeJuice,Dog,LuckyStrike,Ivory
5,Japanese,Coffee,Zebra,Parliaments,Green


## Checking the Uniqueness of the Solution

Given a set of unit clauses $U$, the function $\texttt{checkUniqueness}(U)$ returns a clause that is the negation of the set $U$.

In [144]:
def negateSolution(UnitClauses):
    return { dp.complement(arb(unit)) for unit in UnitClauses }

In [145]:
negateSolution({ frozenset({'a'}), frozenset({('¬', 'b')}) }) 

{('¬', 'a'), 'b'}

In [146]:
def checkUniqueness(Solution, Clauses):
    negation = negateSolution(Solution)
    Clauses.add(frozenset(negation))
    alternative = dp.solve(Clauses, set())
    if alternative == { frozenset() }:
        print("Well done: The solution is unique!")
    else:
        print("ERROR: The solution is not unique!")

In [147]:
checkUniqueness(Solution, Clauses)

Well done: The solution is unique!
